# PYNQ Microblaze

Let's take a look inside the [BaseOverlay class](https://github.com/Xilinx/PYNQ/blob/master/boards/Pynq-Z1/base/base.py) which corresponds to the Overlay below: 

![](http://pynq.readthedocs.io/en/v2.0/_images/pynqz1_base_overlay.png)

Let's load the base overlay and check the IO Processors: iop_arduino, iop_pmoda, and iop_pmodb

In [21]:
import pynq
from pynq.overlays.base import BaseOverlay

ol = BaseOverlay("base.bit")

In [22]:
ol.ip_dict

{'audio_direct_0': {'addr_range': 65536,
  'driver': pynq.lib.audio.AudioDirect,
  'fullpath': 'audio_direct_0',
  'gpio': {'sel_direct': {'index': 3,
    'pins': {'audio_direct_0/sel_direct', 'audio_path_sel/Dout'},
    'state': None}},
  'interrupts': {},
  'phys_addr': 1136656384,
  'state': None,
  'type': 'xilinx.com:user:audio_direct:1.1'},
 'btns_gpio': {'addr_range': 65536,
  'driver': pynq.lib.axigpio.AxiGPIO,
  'fullpath': 'btns_gpio',
  'gpio': {},
  'interrupts': {'ip2intc_irpt': {'controller': 'system_interrupts',
    'fullpath': 'btns_gpio/ip2intc_irpt',
    'index': 11}},
  'phys_addr': 1092681728,
  'state': None,
  'type': 'xilinx.com:ip:axi_gpio:2.0'},
 'iop_arduino/mb_bram_ctrl': {'addr_range': 65536,
  'fullpath': 'iop_arduino/mb_bram_ctrl',
  'gpio': {},
  'interrupts': {},
  'phys_addr': 1140850688,
  'state': None,
  'type': 'xilinx.com:ip:axi_bram_ctrl:4.0'},
 'iop_pmoda/mb_bram_ctrl': {'addr_range': 65536,
  'fullpath': 'iop_pmoda/mb_bram_ctrl',
  'gpio': {},
 

Some PMODs already have drivers: [list of plug and play PMODs](/tree/base/pmod)

In [26]:
from pynq.lib import Pmod_OLED
pmod_oled = Pmod_OLED(ol.PMODA)

In [27]:
pmod_oled.clear()
pmod_oled.write('Welcome to \n\nPYNQ!')

Let's take a look at it's code on [pynq github](https://github.com/Xilinx/PYNQ/tree/master/pynq/lib/pmod)

## Hello World on Microblaze
IO prrocessors can also be used for running small applications.

[IPYthon magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) allow the execution of Non-Python code in a Python cell.

In [2]:
from pynq.lib import MicroblazeLibrary

In [ ]:
%%microblaze ol.PMODA
#include <pyprintf.h>

int mb_print(){
    pyprintf("Hello World!");
    return 0;
}


In [ ]:
mb_print()

## Task 1
Here you learn how to use provided python libraries.

Using the source code for [RGBLED](https://github.com/Xilinx/PYNQ/blob/master/pynq/lib/rgbled.py) write a for loop to change the coloring of your rgb leds between red, blue, and white similar to a police light bar. You can use *time.sleep* as delay between flashes.

In [ ]:
import time
# write code for RGBLED

## Task 2
Write a microblaze code to calculate factorial of an input on PL. Your result should be same as the python code output.

In [ ]:
%%microblaze ol.PMODB

int mb_fact(int in){
//write code here
}

In [ ]:
inp = 10

print('out = {}'.format(mb_fact(inp))) # your function is called here, note the function prototype

ff=1
for i in range(1,inp+1):
    ff = ff*i
print('ans = {}'.format(ff))

## Task 3
Use the [source code](https://github.com/Xilinx/PYNQ/blob/master/pynq/lib/pmod/pmod_oled/src/pmod_oled.c) for OLED_PMOD to learn how to use the GPIO pins.

Write two microblaze functions: *mba_send(v)* and *mbb_recv()* to send 1-bit accross the two PMODs *A* and *B*.

You need to connect pin 0 of the two PMODs using a jumper wire.


In [7]:
%%microblaze ol.PMODA
#include <gpio.h>

int mba_send(int v){
//write code here
}

In [15]:
%%microblaze ol.PMODB
#include <gpio.h>

int mbb_recv(){
//write code here
}

In [19]:
mba_send(1)
print(mbb_recv())

1


## Task 4

Read voltages from the Pmod_AD2. **Since there are only 2 Pmod_AD2. Write the code to the best of your ability and ask one of the TA's to check before testing on the Pmod_AD2.**

* The first part of this task is to only use the pynq python libraries. You can reference the API [here](https://pynq.readthedocs.io/en/v2.0/pynq_package/pynq.lib/pynq.lib.pmod.html#). *Remember to close the pmod instance*

In [20]:
# write code here

* For the second part of the task, we'll see how to communicate with the Pmod_AD2 by programming the microblaze in C. While this is more work, most of the PMODS we have available do not have python libraries pre-built so you'll need to search through the documentation and program the microblaze in order to use them.

[microblaze libraries for pynq](https://pynq.readthedocs.io/en/latest/pynq_libraries/pynqmb_reference.html)

[Pmod_AD2 reference manual](https://reference.digilentinc.com/reference/pmod/pmodad2/reference-manual)

In [ ]:
%%microblaze base.PMODA
#include <i2c.h>
#include <pyprintf.h>

//TODO: Find the pmod_ad2 address value
#define AD2IICAddr         <#>

//Configuration
#define CH3                 7
#define CH2                 6
#define CH1                 5
#define CH0                 4
#define REF_SEL             3
#define FLTR                2
#define BitTrialDelay       1
#define SampleDelay         0

#define BitMask             0xFFF

float read_i2c() {
    
    //TODO: open a new i2c device
    
    unsigned char WriteBuffer[1];    
    unsigned char cfgValue = (1 << CH3)   |
               (1 << CH2)                 |
               (1 << CH1)                 |
               (1 << CH0)                 |
               (0 << REF_SEL)             |
               (0 << FLTR)                |
               (0 << BitTrialDelay)       |
               (0 << SampleDelay);
    WriteBuffer[0]=cfgValue;
    //TODO: write the configuration to the pmod (1 byte)
    
    //Receiving data.
    unsigned char rcvbuffer[2];
    int rxData;
    //TODO: read from the pmod and format the raw data (2 bytes)
    // The first byte is MSB, while the second byte is LSB
    rxData = //;
    
    
    //Format as a voltage
    int raw = (rxData & BitMask);    
    return (float)(raw * 2.00 / 4096.0); // 2.0 V is the reference voltage for the AD2 Pmod.
}